# Analysis of the Loan Data from Prosper by Barbara Stempien

## Table of Contents¶

* [Introduction](#Introduction)
* [Data Wrangling](#Data-Wrangling)
* [Exploratory Data Analysis](#Exploratory-Data-Analysis)
* [Conclusions](#Conclusions)


## Introduction

Prosper loan data set contains 113,937 loans with 81 variables on each loan, including loan amount, borrower rate (or interest rate), current loan status, borrower income, borrower employment status, borrower credit history, and the latest payment information.

In [94]:
# load packages
library(ggplot2)
library(dplyr)
library(gridExtra)
library(GGally)
library(scales)
library(memisc)
library(repr)
library(RColorBrewer)
library(bitops)
library(RCurl)

In [95]:
# load the prosper loan dataset
loan_dataset <- read.csv('data/prosperLoanData.csv')

In [96]:
# display shape of dataset; rows x columns
dim(loan_dataset)

[1] 113937     81

In [97]:
# display few lines
head(loan_dataset)

ListingKey,ListingNumber,ListingCreationDate,CreditGrade,Term,LoanStatus,ClosedDate,BorrowerAPR,BorrowerRate,LenderYield,...,LP_ServiceFees,LP_CollectionFees,LP_GrossPrincipalLoss,LP_NetPrincipalLoss,LP_NonPrincipalRecoverypayments,PercentFunded,Recommendations,InvestmentFromFriendsCount,InvestmentFromFriendsAmount,Investors
1021339766868145413AB3B,193129,2007-08-26 19:09:29.263000000,C,36,Completed,2009-08-14 00:00:00,0.16516,0.1580,0.1380,...,-133.18,0,0,0,0,1,0,0,0,258
10273602499503308B223C1,1209647,2014-02-27 08:28:07.900000000,,36,Current,,0.12016,0.0920,0.0820,...,0.00,0,0,0,0,1,0,0,0,1
0EE9337825851032864889A,81716,2007-01-05 15:00:47.090000000,HR,36,Completed,2009-12-17 00:00:00,0.28269,0.2750,0.2400,...,-24.20,0,0,0,0,1,0,0,0,41
0EF5356002482715299901A,658116,2012-10-22 11:02:35.010000000,,36,Current,,0.12528,0.0974,0.0874,...,-108.01,0,0,0,0,1,0,0,0,158
0F023589499656230C5E3E2,909464,2013-09-14 18:38:39.097000000,,36,Current,,0.24614,0.2085,0.1985,...,-60.27,0,0,0,0,1,0,0,0,20
0F05359734824199381F61D,1074836,2013-12-14 08:26:37.093000000,,60,Current,,0.15425,0.1314,0.1214,...,-25.33,0,0,0,0,1,0,0,0,1


## Data Wrangling

In [98]:
# subset the dataframe to the following columns
subset <- c("ListingKey",
            "LoanStatus",
            "ListingCreationDate",
            "ClosedDate",
            "ListingCategory..numeric.",
            "Term",
            "BorrowerRate",
            "LoanOriginalAmount",
            "MonthlyLoanPayment",
            "BorrowerState",
            "IsBorrowerHomeowner",  
            "Occupation",
            "EmploymentStatus",
            "EmploymentStatusDuration",
            "StatedMonthlyIncome",
            "DebtToIncomeRatio",
            "ProsperRating..numeric.",
            "ProsperScore",           
            "CreditScoreRangeLower",
            "CreditScoreRangeUpper",
            "OpenCreditLines",
            "TotalCreditLinespast7years",
            "OpenRevolvingAccounts",
            "OpenRevolvingMonthlyPayment",
            "CurrentDelinquencies",
            "AmountDelinquent",
            "DelinquenciesLast7Years",
            "PublicRecordsLast10Years",
            "RevolvingCreditBalance",
            "BankcardUtilization",
            "TotalProsperLoans",
            "ProsperPrincipalBorrowed",
            "ProsperPrincipalOutstanding",
            "ProsperPaymentsLessThanOneMonthLate",
            "ProsperPaymentsOneMonthPlusLate"
           )

In [99]:
# subset dataframe
df <- loan_dataset[subset]

In [100]:
# replace NAs with 0
df[is.na(df)] <- 0

In [101]:
# display type of each variable
str(df)

'data.frame':	113937 obs. of  35 variables:
 $ ListingKey                         : Factor w/ 113066 levels "00003546482094282EF90E5",..: 7180 7193 6647 6669 6686 6689 6699 6706 6687 6687 ...
 $ LoanStatus                         : Factor w/ 12 levels "Cancelled","Chargedoff",..: 3 4 3 4 4 4 4 4 4 4 ...
 $ ListingCreationDate                : Factor w/ 113064 levels "2005-11-09 20:44:28.847000000",..: 14184 111894 6429 64760 85967 100310 72556 74019 97834 97834 ...
 $ ClosedDate                         : Factor w/ 2803 levels "","2005-11-25 00:00:00",..: 1138 1 1263 1 1 1 1 1 1 1 ...
 $ ListingCategory..numeric.          : int  0 2 0 16 2 1 1 2 7 7 ...
 $ Term                               : int  36 36 36 36 36 60 36 36 36 36 ...
 $ BorrowerRate                       : num  0.158 0.092 0.275 0.0974 0.2085 ...
 $ LoanOriginalAmount                 : int  9425 10000 3001 10000 15000 15000 3000 10000 10000 10000 ...
 $ MonthlyLoanPayment                 : num  330 319 123 321 564 ...
 $ B

In [102]:
# convert Listing Creation Date variable to date format
df$ListingCreationDate <- as.Date(df$ListingCreationDate)

In [103]:
# Term variable 
table(df$Term)

# convert term to ordered factor with 3 levels
df$Term <- factor(df$Term, levels=c(12,36,60), ordered=TRUE)


   12    36    60 
 1614 87778 24545 

In [104]:
# convert Closed Date variable to date format
df$ClosedDate <- as.Date(df$ClosedDate)

In [105]:
# Prosper Rating variable 
table(df$ProsperRating)

# convert Prosper Rating to ordered factor with 8 levels
df$ProsperRating..numeric. <- factor(df$ProsperRating..numeric., levels=c(0,1,2,3,4,5,6,7), ordered=TRUE)


    0     1     2     3     4     5     6     7 
29084  6935  9795 14274 18345 15581 14551  5372 

In [106]:
# Prosper Score variable 
table(df$ProsperScore)

# convert Prosper Score to ordered factor with 12 levels
df$ProsperScore <- factor(df$ProsperScore, levels=c(0,1,2,3,4,5,6,7,8,9,10,11), ordered=TRUE)


    0     1     2     3     4     5     6     7     8     9    10    11 
29084   992  5766  7642 12595  9813 12278 10597 12053  6911  4750  1456 

In [107]:
# Listing Category variable
table(df$ListingCategory..numeric.)

# add column with Listing Category labels based on the documentation
df$ListingCategory <- factor(df$ListingCategory..numeric., labels=c('Not Available',
                                                                    'Debt Consolidation',
                                                                    'Home Improvement',
                                                                    'Business',
                                                                    'Personal Loan',
                                                                    'Student Use',
                                                                    'Auto',
                                                                    'Other',
                                                                    'Baby&Adoption',
                                                                    'Boat',
                                                                    'Cosmetic Procedure',
                                                                    'Engagement Ring',
                                                                    'Green Loans',
                                                                    'Household Expenses',
                                                                    'Large Purchases',
                                                                    'Medical/Dental',
                                                                    'Motorcycle',
                                                                    'RV',
                                                                    'Taxes',
                                                                    'Vacation',
                                                                    'Wedding Loans'))

# move ListingCategory column next to the ListingCategory..numeric.
df <- df[c(1:5,36,6:35)]


    0     1     2     3     4     5     6     7     8     9    10    11    12 
16965 58308  7433  7189  2395   756  2572 10494   199    85    91   217    59 
   13    14    15    16    17    18    19    20 
 1996   876  1522   304    52   885   768   771 

In [108]:
# Is Borrower Homeowner variable
table(df$IsBorrowerHomeowner)

# convert Is Borrower Homeowner to logical type
df$IsBorrowerHomeowner <- as.logical(df$IsBorrowerHomeowner)


False  True 
56459 57478 

In [109]:
# display type of each variable
str(df2)

'data.frame':	113937 obs. of  36 variables:
 $ ListingKey                         : Factor w/ 113066 levels "00003546482094282EF90E5",..: 7180 7193 6647 6669 6686 6689 6699 6706 6687 6687 ...
 $ LoanStatus                         : Factor w/ 12 levels "Cancelled","Chargedoff",..: 3 4 3 4 4 4 4 4 4 4 ...
 $ ListingCreationDate                : Date, format: "2007-08-26" "2014-02-27" ...
 $ ClosedDate                         : Date, format: "2009-08-14" NA ...
 $ ListingCategory..numeric.          : int  0 2 0 16 2 1 1 2 7 7 ...
 $ ListingCategory                    : Factor w/ 21 levels "Not Available",..: 1 3 1 17 3 2 2 3 8 8 ...
 $ Term                               : Ord.factor w/ 3 levels "12"<"36"<"60": 2 2 2 2 2 3 2 2 2 2 ...
 $ BorrowerRate                       : num  0.158 0.092 0.275 0.0974 0.2085 ...
 $ LoanOriginalAmount                 : int  9425 10000 3001 10000 15000 15000 3000 10000 10000 10000 ...
 $ MonthlyLoanPayment                 : num  330 319 123 321 564 ...
 $ 

In [110]:
# display few lines of new dataset
head(df)

ListingKey,LoanStatus,ListingCreationDate,ClosedDate,ListingCategory..numeric.,ListingCategory,Term,BorrowerRate,LoanOriginalAmount,MonthlyLoanPayment,...,AmountDelinquent,DelinquenciesLast7Years,PublicRecordsLast10Years,RevolvingCreditBalance,BankcardUtilization,TotalProsperLoans,ProsperPrincipalBorrowed,ProsperPrincipalOutstanding,ProsperPaymentsLessThanOneMonthLate,ProsperPaymentsOneMonthPlusLate
1021339766868145413AB3B,Completed,2007-08-26,2009-08-14,0,Not Available,36,0.1580,9425,330.43,...,472,4,0,0,0.00,0,0,0.0,0,0
10273602499503308B223C1,Current,2014-02-27,NA,2,Home Improvement,36,0.0920,10000,318.93,...,0,0,1,3989,0.21,0,0,0.0,0,0
0EE9337825851032864889A,Completed,2007-01-05,2009-12-17,0,Not Available,36,0.2750,3001,123.32,...,0,0,0,0,0.00,0,0,0.0,0,0
0EF5356002482715299901A,Current,2012-10-22,NA,16,Motorcycle,36,0.0974,10000,321.45,...,10056,14,0,1444,0.04,0,0,0.0,0,0
0F023589499656230C5E3E2,Current,2013-09-14,NA,2,Home Improvement,36,0.2085,15000,563.97,...,0,0,0,6193,0.81,1,11000,9947.9,0,0
0F05359734824199381F61D,Current,2013-12-14,NA,1,Debt Consolidation,60,0.1314,15000,342.37,...,0,0,0,62999,0.39,0,0,0.0,0,0


## Exploratory Data Analysis

In [111]:
# change size of plots
options(repr.plot.width=8, repr.plot.height=4)